# XLSX Extraction: Capabilities Walkthrough

This notebook demonstrates the XLSX extraction pipeline — from raw Excel files to
clean `StructuredTable` objects ready for interpretation.

**Story arc:** Basic Extraction → Merged Headers → Multi-Table Detection (XH1) →
Title Detection (XH2) → Hidden Content Filtering (XH3) → Number Format Hints (XH4) →
Markdown Rendering

All examples use synthetic fixtures from `inputs/xlsx/synthetic/` — no external
documents or API keys required.

In [ ]:
# Setup — imports and fixture paths
from pathlib import Path

SYNTH = Path("inputs/xlsx/synthetic")

# P1: Tidy Analyst
SIMPLE    = SYNTH / "p1_simple_financial.xlsx"
INVENTORY = SYNTH / "p1_inventory_mixed.xlsx"
TIMESERIES = SYNTH / "p1_time_series.xlsx"
MULTISHEET = SYNTH / "p1_multi_sheet.xlsx"

# P2: The Merger
BUDGET    = SYNTH / "p2_budget_merged.xlsx"
DEEP      = SYNTH / "p2_deep_hierarchy.xlsx"
CROSSTAB  = SYNTH / "p2_cross_tab.xlsx"
IRREGULAR = SYNTH / "p2_irregular_merges.xlsx"

# P3: The Multi-Tasker
TWO_TABLES = SYNTH / "p3_two_tables_blank_rows.xlsx"
TITLED     = SYNTH / "p3_tables_with_titles.xlsx"
FOOTNOTES  = SYNTH / "p3_tables_with_footnotes.xlsx"
SIDE_BY_SIDE = SYNTH / "p3_side_by_side.xlsx"

# P4: The Formatter
KPI        = SYNTH / "p4_kpi_dashboard.xlsx"
BANDED     = SYNTH / "p4_banded_report.xlsx"
HIDDEN     = SYNTH / "p4_hidden_columns.xlsx"
DATEFORMATS = SYNTH / "p4_date_formats.xlsx"

from docpact.xlsx_extractor import (
    extract_tables_from_xlsx,
    xlsx_to_markdown,
)

print(f"Fixtures: {len(list(SYNTH.glob('*.xlsx')))} files in {SYNTH}")

---
## Section 1: Basic Extraction (P1 — The Tidy Analyst)

Simple, clean spreadsheets with one table per sheet. No merges, no tricks.
These validate that the basics work: header detection, column naming, data extraction.

In [ ]:
# Simple financial table: 5 columns, 10 rows
tables = extract_tables_from_xlsx(SIMPLE)
t = tables[0]

print(f"Tables found: {len(tables)}")
print(f"Columns: {t.column_names}")
print(f"Data rows: {len(t.data)}")
print(f"Source format: {t.source_format}")
print(f"Sheet: {t.metadata['sheet_name']}")
print()
print("First 5 rows:")
for row in t.data[:5]:
    print(f"  {row}")

In [ ]:
# Mixed types: text, numbers, dates, enum values
tables = extract_tables_from_xlsx(INVENTORY)
t = tables[0]

print(f"Columns ({len(t.column_names)}): {t.column_names}")
print(f"Data rows: {len(t.data)}")
print(f"Format hints: {t.metadata.get('format_hints', {})}")
print()
print("Sample rows (note date in column 4, enum in column 5):")
for row in t.data[:4]:
    print(f"  {row}")

In [ ]:
# Multi-sheet workbook: 3 sheets with different structures
tables = extract_tables_from_xlsx(MULTISHEET)

print(f"Total tables: {len(tables)}")
for t in tables:
    print(f"  Sheet '{t.metadata['sheet_name']}': "
          f"{len(t.column_names)} cols x {len(t.data)} rows — {t.column_names}")

In [ ]:
# Sheet filtering: process only specific sheets
targets_only = extract_tables_from_xlsx(MULTISHEET, sheets=["Targets"])
print(f"Filtered to 'Targets': {len(targets_only)} table")
print(f"Columns: {targets_only[0].column_names}")

# Also works by index
by_index = extract_tables_from_xlsx(MULTISHEET, sheets=[2])
print(f"\nSheet index 2: '{by_index[0].metadata['sheet_name']}'")
print(f"Columns: {by_index[0].column_names}")

---
## Section 2: Merged Headers (P2 — The Merger)

Real-world Excel files use merged cells to create hierarchical headers:
"Revenue" spans over "Q1" and "Q2". The extractor expands these into compound
column names using ` / ` separators, matching the DOCX extractor convention.

Header detection uses three methods (taking the maximum):
1. **Merge-based**: Last horizontal merge row + type continuation
2. **Type-pattern**: Consecutive all-string rows from the top (TH2)
3. **Span-count**: Bottom-up pattern analysis (H7)

In [ ]:
# 2-level header: Revenue/Cost each span 2 columns over Q1/Q2
tables = extract_tables_from_xlsx(BUDGET)
t = tables[0]

print(f"Compound headers ({len(t.column_names)} columns):")
for i, col in enumerate(t.column_names):
    print(f"  [{i}] {col}")
print(f"\nData rows: {len(t.data)}")
for row in t.data[:3]:
    print(f"  {row}")

In [ ]:
# 3-level header: Year > Quarter > Month
tables = extract_tables_from_xlsx(DEEP)
t = tables[0]

print(f"Deep hierarchy: {len(t.column_names)} columns, {len(t.data)} data rows")
print("\nColumn names (first 7):")
for i, col in enumerate(t.column_names[:7]):
    print(f"  [{i:2d}] {col}")
print(f"  ...")
print(f"  [{len(t.column_names)-1:2d}] {t.column_names[-1]}")

In [ ]:
# Cross-tab: merged row AND column labels
tables = extract_tables_from_xlsx(CROSSTAB)
t = tables[0]

print(f"Cross-tab: {len(t.column_names)} columns")
for i, col in enumerate(t.column_names):
    print(f"  [{i}] {col}")
print(f"\nData ({len(t.data)} rows):")
for row in t.data:
    print(f"  {row}")

In [ ]:
# Irregular merges with gaps — graceful handling
tables = extract_tables_from_xlsx(IRREGULAR)
t = tables[0]

print(f"Irregular merges: {len(t.column_names)} columns, {len(t.data)} rows")
for i, col in enumerate(t.column_names):
    print(f"  [{i}] {col}")

---
## Section 3: Multi-Table Detection — XH1

**The challenge**: A single worksheet may contain multiple tables, separated by blank
rows, title rows, footnotes, or blank columns (side-by-side layout).

**XH1 heuristic**: Scan for runs of ≥2 consecutive blank rows (vertical separator)
or ≥2 blank columns (horizontal separator). Each non-empty rectangle ≥2×2 becomes
a candidate table.

**Rationale**: One blank row is breathing room (subtotals, section breaks). Two+ blank
rows mean "these are separate things." This matches human visual perception.

In [ ]:
# Two tables separated by 3 blank rows on one sheet
tables = extract_tables_from_xlsx(TWO_TABLES)

print(f"Tables found: {len(tables)} (on 1 sheet)")
for i, t in enumerate(tables):
    print(f"\n--- Table {i} ---")
    print(f"  Columns: {t.column_names}")
    print(f"  Data rows: {len(t.data)}")
    print(f"  First row: {t.data[0]}")

In [ ]:
# Side-by-side tables: 2 tables separated by blank columns
tables = extract_tables_from_xlsx(SIDE_BY_SIDE)

print(f"Side-by-side tables: {len(tables)}")
for i, t in enumerate(tables):
    print(f"\n--- Table {i} (cols {t.metadata['table_index']}) ---")
    print(f"  Columns: {t.column_names}")
    print(f"  Data rows: {len(t.data)}")
    for row in t.data[:2]:
        print(f"    {row}")

In [ ]:
# Tables with footnotes between them — noise handling
tables = extract_tables_from_xlsx(FOOTNOTES)

print(f"Tables found: {len(tables)}")
for i, t in enumerate(tables):
    print(f"\n--- Table {i} ---")
    print(f"  Columns: {t.column_names}")
    print(f"  Data rows: {len(t.data)}")

# Note: footnotes appear in table 0's data rows since they're
# only 0 blank rows from the data. Downstream processing filters them.
print("\nTable 0 last 3 rows (includes footnotes):")
for row in tables[0].data[-3:]:
    print(f"  {row}")

---
## Section 4: Title Detection — XH2

**XH2 heuristic**: If a table's first row has exactly one non-empty cell and the
table has multi-row headers, that row is a title — not a column header.

The title is extracted into `metadata["title"]` and excluded from compound header
construction. This mirrors DOCX's DH3 (title row detection).

In [ ]:
# Tables with title rows above them
tables = extract_tables_from_xlsx(TITLED)

print(f"Tables found: {len(tables)}")
for i, t in enumerate(tables):
    title = t.metadata.get('title', '(none)')
    print(f"\n--- Table {i}: title = {title!r} ---")
    print(f"  Columns: {t.column_names}")
    print(f"  Data rows: {len(t.data)}")
    # Title should NOT appear in column names
    assert title not in t.column_names, "Title leaked into column names!"

---
## Section 5: Hidden Content Filtering — XH3

**XH3 heuristic**: Authors sometimes hide columns (intermediate calculations,
sensitive data) or rows. These are present in the file but not visible in the
spreadsheet application.

By default, `extract_tables_from_xlsx()` excludes hidden rows and columns.
This can be disabled with `filter_hidden=False`.

In [ ]:
# Hidden columns: Tax Rate (C) and Tax Amount (D) are hidden
# With filtering (default)
tables_filtered = extract_tables_from_xlsx(HIDDEN)
# Without filtering
tables_all = extract_tables_from_xlsx(HIDDEN, filter_hidden=False)

print("With hidden filtering (default):")
print(f"  Columns ({len(tables_filtered[0].column_names)}): {tables_filtered[0].column_names}")
print(f"  First row: {tables_filtered[0].data[0]}")

print("\nWithout hidden filtering:")
print(f"  Columns ({len(tables_all[0].column_names)}): {tables_all[0].column_names}")
print(f"  First row: {tables_all[0].data[0]}")

print(f"\n→ Hidden filtering removed {len(tables_all[0].column_names) - len(tables_filtered[0].column_names)} columns")

---
## Section 6: Number Format Hints — XH4

**XH4 heuristic**: Excel cells have a `number_format` property that distinguishes
dates from numbers from currencies. The extractor inspects this and provides
format hints in the metadata.

Format hints are informational — they don't change the extracted values, but
downstream consumers can use them for type coercion.

In [ ]:
# Date formats: same date in 4 regional formats
tables = extract_tables_from_xlsx(DATEFORMATS)
t = tables[0]

print(f"Columns: {t.column_names}")
print(f"Format hints: {t.metadata.get('format_hints', {})}")
print()
print("Data (same dates, different original formats):")
for row in t.data:
    print(f"  {row}")

In [ ]:
# Time series: date column with format hint
tables = extract_tables_from_xlsx(TIMESERIES)
t = tables[0]

fmt = t.metadata.get('format_hints', {})
print(f"Column 0 ('{t.column_names[0]}'): format hint = {fmt.get(0, 'none')}")
print(f"\nFirst 5 rows:")
for row in t.data[:5]:
    print(f"  {row}")

---
## Section 7: Visual Style Heuristics

When `extract_styles=True` (default), the extractor captures per-cell visual info:
fill colors, borders, bold/italic fonts. These feed into visual heuristics:

- **VH1**: Grid borders (>50% of cells bordered)
- **VH2**: Header fill rows (first N rows with consistent fill color)
- **VH3**: Zebra striping (alternating row colors in data zone)
- **VH4**: Section separators (rows with thick bottom borders)

Visual info serves as cross-validation for text-based heuristics.

In [ ]:
# KPI dashboard: headers with fill, status column with conditional colors
from docpact.xlsx_extractor import _extract_table_from_range, _find_data_bounds, _detect_table_regions
from openpyxl import load_workbook

wb = load_workbook(KPI, data_only=True)
ws = wb.active
bounds = _find_data_bounds(ws)
regions = _detect_table_regions(ws, bounds)
et = _extract_table_from_range(ws, *regions[0])
wb.close()

vis = et.visual
print("Visual heuristics for KPI dashboard:")
print(f"  VH1 Grid borders: {vis.has_grid_borders}")
print(f"  VH2 Header fill rows: {vis.header_fill_rows}")
print(f"  VH3 Zebra pattern: {vis.zebra_pattern}")
print(f"  VH4 Separator rows: {vis.separator_rows}")

# Check specific cell styles
print("\nHeader cell styles (row 0):")
for ci, style in enumerate(et.styles[0]):
    print(f"  Col {ci}: bold={style.font_bold}, fill={style.fill_color is not None}")

In [ ]:
# Banded report: zebra rows, bold headers, bordered grid
wb = load_workbook(BANDED, data_only=True)
ws = wb.active
bounds = _find_data_bounds(ws)
regions = _detect_table_regions(ws, bounds)
et = _extract_table_from_range(ws, *regions[0])
wb.close()

vis = et.visual
print("Visual heuristics for banded report:")
print(f"  VH1 Grid borders: {vis.has_grid_borders}")
print(f"  VH2 Header fill rows: {vis.header_fill_rows}")
print(f"  VH3 Zebra pattern: {vis.zebra_pattern}")
print(f"  VH4 Separator rows: {vis.separator_rows}")

---
## Section 8: Markdown Rendering

`xlsx_to_markdown()` is a convenience function that extracts tables and renders
them as pipe-table markdown — the same format consumed by `interpret_table()` and
`classify_tables()`. This makes the XLSX output compatible with the entire
downstream pipeline.

In [ ]:
# Markdown rendering of the budget table (merged headers)
md = xlsx_to_markdown(BUDGET)
print(md)

In [ ]:
# Multi-sheet rendering
md = xlsx_to_markdown(MULTISHEET)
print(md)

---
## Section 9: Full Fixture Survey

Run all 16 synthetic fixtures through the extractor and display a summary.

In [ ]:
# Survey all 16 fixtures
print(f"{'Fixture':<35s} {'Tables':>6s} {'Cols':>5s} {'Rows':>5s} {'Title':>20s} {'Fmt Hints':>10s}")
print("-" * 90)

total_tables = 0
total_rows = 0

for path in sorted(SYNTH.glob("*.xlsx")):
    tables = extract_tables_from_xlsx(path)
    for i, t in enumerate(tables):
        meta = t.metadata or {}
        title = meta.get('title', '')
        fmt = meta.get('format_hints', {})
        fmt_str = ', '.join(f"c{k}={v}" for k, v in fmt.items()) if fmt else ''
        label = f"{path.stem}" if i == 0 else f"  (table {i})"
        print(f"{label:<35s} {len(tables) if i == 0 else '':>6} "
              f"{len(t.column_names):>5d} {len(t.data):>5d} "
              f"{title:>20s} {fmt_str:>10s}")
        total_tables += 1 if i == 0 else 0
        total_rows += len(t.data)

# Count tables (including multi-table sheets)
all_tables = sum(
    len(extract_tables_from_xlsx(p))
    for p in SYNTH.glob("*.xlsx")
)
print(f"\nTotal: {all_tables} tables from {total_tables} files, {total_rows} data rows")

---
## Summary

| Heuristic | Name | What It Does |
|-----------|------|-------------|
| XH1 | Blank Row/Column Boundary | Splits multi-table sheets into separate tables |
| XH2 | Title Row Detection | Extracts single-cell title rows into metadata |
| XH3 | Hidden Content Filtering | Excludes hidden rows and columns |
| XH4 | Number Format Hints | Provides date/currency/percentage type hints |
| VH1-VH4 | Visual Heuristics | Grid detection, header fills, zebra rows, separators |

### Header Detection — Layered Approach

| Method | Source | Best For |
|--------|--------|----------|
| Merge-based | `ws.merged_cells.ranges` | Multi-row merged headers |
| Type-pattern (TH2) | All-string rows from top | Simple flat tables |
| Span-count (H7) | Bottom-up cell-count analysis | Irregular structures |
| Visual fill (VH2) | Header background colors | Styled spreadsheets |

The extractor takes the **maximum** across all methods, ensuring robust detection
across diverse spreadsheet styles.

### Output Format

`extract_tables_from_xlsx()` returns `list[StructuredTable]` — the same type used
by PDF and DOCX extractors. All downstream processing (classification, interpretation,
serialization) is format-agnostic.